In [2]:
!pip install -r requirements.txt

  Using cached torch-2.2.2-cp310-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp310-cp310-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.2.2-cp310-cp310-macosx_10_13_x86_64.whl.metadata (6.4 kB)
  Using cached numpy-1.24.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (5.6 kB)
  Using cached matplotlib-3.10.8-cp310-cp310-macosx_10_12_x86_64.whl.metadata (52 kB)
  Using cached tqdm-4.67.3-py3-none-any.whl.metadata (57 kB)
  Using cached torchviz-0.0.3-py3-none-any.whl.metadata (2.1 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2026.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Usi

In [3]:
import time
import torch
import numpy as np
import pandas as pd
import utils
import model

In [7]:
MODEL_PATH = "model.pt"
DATA_PATH = "../data/scheduling_data_out_ab_nginx.csv"

BATCH_SIZE = 984
HIDDEN_DIM = 51
FUTURE = 1000

device = utils.DEVICE

In [8]:
def task_prediction_accuracy(pred, target):
    """
    Computes classification accuracy of next-task prediction.
    """
    pred_task = pred[:, :, 1:]
    target_task = target[:, :, 1:]

    pred_labels = torch.argmax(pred_task, dim=2)
    true_labels = torch.argmax(target_task, dim=2)

    correct = (pred_labels == true_labels).float()
    return correct.mean().item()

In [10]:
print("Loading dataset...")
df = pd.read_csv(DATA_PATH)
df = utils.preprocess_data(df=df)
data = df.to_numpy(dtype=np.float32)

data_dim = data.shape[1]
total = data.shape[0]
remainder = total % BATCH_SIZE
data = data[:total - remainder, :]

data = data.reshape(BATCH_SIZE, data_dim, -1)
data = np.transpose(data, axes=(0, 2, 1))

train_x, train_y, test_x, test_y = utils.make_training_and_testing_set(
    data, percent_train=85.0
)

test_x = test_x.to(device=device, dtype=torch.float32)
test_y = test_y.to(device=device, dtype=torch.float32)

print("Train shape:", train_x.shape)
print("Test shape:", test_x.shape)

Loading dataset...
Train shape: torch.Size([836, 1171, 28])
Test shape: torch.Size([148, 1171, 28])


In [11]:
print("Loading trained model...")
seq = model.Sequence2(
    in_dim=data_dim,
    out_dim=data_dim,
    hidden_dim=HIDDEN_DIM
).to(device)

seq.load_state_dict(torch.load(MODEL_PATH, map_location=device))
seq.eval()

Loading trained model...


Sequence2(
  (lstm1): LSTMCell(28, 51)
  (lstm2): LSTMCell(51, 51)
  (linear): Linear(in_features=51, out_features=28, bias=True)
)

In [12]:
criterion = torch.nn.MSELoss().to(device)

with torch.no_grad():
    pred = seq(test_x, future=FUTURE)

    loss = criterion(pred[:, :-FUTURE], test_y)
    acc = task_prediction_accuracy(pred[:, :-FUTURE], test_y)

    print("\n===== Prediction Metrics =====")
    print(f"Test MSE Loss: {loss.item():.6f}")
    print(f"Task Prediction Accuracy: {acc * 100:.2f}%")


===== Prediction Metrics =====
Test MSE Loss: 0.017049
Task Prediction Accuracy: 14.68%


In [13]:
print("\nMeasuring inference latency...")

sample = test_x[:1]

# warmup
for _ in range(10):
    _ = seq(sample)

start = time.perf_counter()
for _ in range(100):
    _ = seq(sample)
end = time.perf_counter()

latency = (end - start) / 100

print("\n===== Latency =====")
print(f"Single prediction latency: {latency * 1e6:.3f} µs")


Measuring inference latency...

===== Latency =====
Single prediction latency: 323430.721 µs
